In [ ]:
!pip install requests
!pip install pandas
!pip install numpy
!pip install scikit-learn
!pip install requests 

In [1]:
import pandas as pd
import numpy as np

# Data Preparation

In [2]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [3]:
%%time
df = pd.read_csv("data/PS_20174392719_1491204439457_log.tar.gz", compression="gzip")

CPU times: user 16.9 s, sys: 1.48 s, total: 18.3 s
Wall time: 18.4 s


In [4]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,3,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,3,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,4,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,1,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,3,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [5]:
# Transform type column to be category
df['type'] = df['type'].astype('category')
df['type'] = df['type'].cat.codes

In [6]:
train = df[df['isFraud'] == 0]
test = df[df['isFraud'] == 1]

In [7]:
clean, fraudulent = train_test_split(train, test_size=0.05)
fraudulent = pd.concat([test, fraudulent]).sample(frac=1).reset_index(drop=True)
fraudulent, fraudulent_labels = fraudulent.drop(columns=['nameOrig', 'nameDest', 'isFraud', 'isFlaggedFraud'], axis=1), fraudulent['isFraud']
clean = clean.drop(columns=['nameOrig', 'nameDest', 'isFraud', 'isFlaggedFraud'], axis=1)

In [8]:
clean.head()

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
1484863,141,1,33107.12,0.00,0.0,4245938.03,4279045.14
355012,17,0,279226.33,2512072.07,2791298.4,1417433.03,1138206.70
4442671,323,1,349919.29,509.00,0.0,376458.59,726377.88
2340225,189,1,85566.70,0.00,0.0,338844.19,424410.89
5419481,378,3,11770.45,0.00,0.0,0.00,0.00


# Inference

In [9]:
import requests
import time 

For Autoencoder monitoring use threshold __7362628712__

In [25]:
HOST_ADDRESS = "https://demo.k8s.hydrosphere.io"
# HOST_ADDRESS = "http://localhost"

In [11]:
def send_data(data, labels=None):
    global HOST_ADDRESS, APPLICATION_NAME, SIGNATURE_NAME
    print("Using URL", f"{HOST_ADDRESS}/gateway/applications/{APPLICATION_NAME}/{SIGNATURE_NAME}")
    for index, point in data.iterrows():
        point = [point.values.tolist()]
        print(point)
        
        r = requests.post(
            url=f"{HOST_ADDRESS}/gateway/applications/{APPLICATION_NAME}/{SIGNATURE_NAME}", 
            json={'features': point})
        
        if labels is not None:
            print(r.json(), bool(labels.loc[index]))
        else:
            print(r.json())
        time.sleep(0.6)

### Random Forests

In [17]:
APPLICATION_NAME = "fraud_rf"
SIGNATURE_NAME = "infer"

In [ ]:
send_data(clean)

Using URL https://demo.k8s.hydrosphere.io/gateway/applications/fraud_rf/infer
[[141.0, 1.0, 33107.12, 0.0, 0.0, 4245938.03, 4279045.14]]
{'is_fraud': [False]}
[[17.0, 0.0, 279226.33, 2512072.07, 2791298.4, 1417433.03, 1138206.7]]
{'is_fraud': [False]}
[[323.0, 1.0, 349919.29, 509.0, 0.0, 376458.59, 726377.88]]
{'is_fraud': [False]}
[[189.0, 1.0, 85566.7, 0.0, 0.0, 338844.19, 424410.89]]
{'is_fraud': [False]}
[[378.0, 3.0, 11770.45, 0.0, 0.0, 0.0, 0.0]]
{'is_fraud': [False]}
[[254.0, 0.0, 151322.84, 5743.0, 157065.84, 5470448.42, 5319125.59]]
{'is_fraud': [False]}
[[251.0, 3.0, 496.82, 0.0, 0.0, 0.0, 0.0]]
{'is_fraud': [False]}
[[369.0, 3.0, 1502.51, 0.0, 0.0, 0.0, 0.0]]
{'is_fraud': [False]}
[[39.0, 3.0, 8313.99, 18266.0, 9952.01, 0.0, 0.0]]
{'is_fraud': [False]}
[[514.0, 3.0, 5609.33, 0.0, 0.0, 0.0, 0.0]]
{'is_fraud': [False]}
[[360.0, 1.0, 33682.51, 5055.27, 0.0, 178183.88, 211866.4]]
{'is_fraud': [False]}
[[13.0, 4.0, 995758.97, 25435.0, 0.0, 0.0, 995758.97]]
{'is_fraud': [False]}
[

[[250.0, 3.0, 23984.29, 49761.0, 25776.71, 0.0, 0.0]]
{'is_fraud': [False]}
[[178.0, 0.0, 75692.5, 13404200.33, 13479892.84, 308366.96, 232674.45]]
{'is_fraud': [False]}
[[129.0, 0.0, 263091.89, 4163098.21, 4426190.1, 742707.93, 479616.04]]
{'is_fraud': [False]}
[[590.0, 1.0, 12987.08, 189.0, 0.0, 287974.8, 300961.87]]
{'is_fraud': [False]}
[[285.0, 3.0, 78367.5, 0.0, 0.0, 0.0, 0.0]]
{'is_fraud': [False]}
[[131.0, 3.0, 6072.59, 34739.0, 28666.41, 0.0, 0.0]]
{'is_fraud': [False]}
[[12.0, 3.0, 20411.64, 116.0, 0.0, 0.0, 0.0]]
{'is_fraud': [False]}
[[288.0, 1.0, 423033.67, 0.0, 0.0, 2367224.71, 2790258.39]]
{'is_fraud': [False]}
[[137.0, 0.0, 205558.47, 3741821.32, 3947379.79, 343002.16, 137443.69]]
{'is_fraud': [False]}
[[227.0, 1.0, 18452.93, 47.0, 0.0, 1243884.25, 1262337.18]]
{'is_fraud': [False]}
[[287.0, 3.0, 10483.4, 74347.0, 63863.6, 0.0, 0.0]]
{'is_fraud': [False]}
[[301.0, 3.0, 54837.52, 1851.0, 0.0, 0.0, 0.0]]
{'is_fraud': [False]}
[[188.0, 0.0, 86994.41, 25360454.78, 25447449.

[[229.0, 1.0, 170269.22, 0.0, 0.0, 421008.25, 591277.46]]
{'is_fraud': [False]}
[[203.0, 1.0, 174583.3, 1591.18, 0.0, 533969.21, 708552.51]]
{'is_fraud': [False]}
[[306.0, 0.0, 406266.45, 1542.0, 407808.45, 25395661.54, 24989395.09]]
{'is_fraud': [False]}
[[207.0, 1.0, 245257.84, 0.0, 0.0, 1913591.98, 2158849.82]]
{'is_fraud': [False]}
[[403.0, 0.0, 242363.9, 7077.0, 249440.9, 1910134.63, 1667770.73]]
{'is_fraud': [False]}
[[11.0, 0.0, 24637.49, 5737.0, 30374.49, 103634.0, 78996.51]]
{'is_fraud': [False]}
[[185.0, 1.0, 189682.21, 11612.0, 0.0, 2437694.87, 2627377.08]]
{'is_fraud': [False]}
[[402.0, 1.0, 16561.36, 42633.0, 26071.64, 8130990.47, 8147551.83]]
{'is_fraud': [False]}
[[311.0, 3.0, 2395.16, 27858.61, 25463.45, 0.0, 0.0]]
{'is_fraud': [False]}
[[177.0, 0.0, 265734.25, 16970.0, 282704.25, 0.0, 0.0]]
{'is_fraud': [False]}
[[142.0, 3.0, 10497.54, 52723.0, 42225.46, 0.0, 0.0]]
{'is_fraud': [False]}
[[215.0, 4.0, 1052502.16, 37755.7, 0.0, 6770633.08, 7823135.25]]
{'is_fraud': [Fals

{'is_fraud': [False]}
[[403.0, 3.0, 6856.7, 163287.22, 156430.52, 0.0, 0.0]]
{'is_fraud': [False]}
[[182.0, 3.0, 3910.01, 0.0, 0.0, 0.0, 0.0]]
{'is_fraud': [False]}
[[236.0, 1.0, 152235.81, 0.0, 0.0, 4794569.37, 4946805.18]]
{'is_fraud': [False]}
[[182.0, 3.0, 3403.38, 59227.0, 55823.62, 0.0, 0.0]]
{'is_fraud': [False]}
[[213.0, 1.0, 47793.21, 10824.0, 0.0, 78487.66, 126280.87]]
{'is_fraud': [False]}
[[371.0, 1.0, 212267.33, 209731.0, 0.0, 0.0, 212267.33]]
{'is_fraud': [False]}
[[374.0, 1.0, 126344.95, 0.0, 0.0, 1794180.61, 1920525.56]]
{'is_fraud': [False]}
[[13.0, 3.0, 26871.21, 58703.0, 31831.79, 0.0, 0.0]]
{'is_fraud': [False]}
[[330.0, 1.0, 24136.08, 0.0, 0.0, 6334928.73, 6359064.8]]
{'is_fraud': [False]}
[[209.0, 3.0, 6151.11, 0.0, 0.0, 0.0, 0.0]]
{'is_fraud': [False]}
[[280.0, 0.0, 148959.26, 442369.03, 591328.28, 395279.75, 246320.49]]
{'is_fraud': [False]}
[[278.0, 1.0, 101719.28, 0.0, 0.0, 147655.82, 249375.1]]
{'is_fraud': [False]}
[[250.0, 1.0, 578784.72, 106418.0, 0.0, 845

In [29]:
send_data(fraudulent, fraudulent_labels)

Using URL https://demo.k8s.hydrosphere.io/gateway/applications/fraud_rf/infer
[[238.0, 3.0, 1550.77, 4853.33, 3302.56, 0.0, 0.0]]
{'is_fraud': [False]} False
[[399.0, 1.0, 50925.07, 454234.57, 403309.5, 10640025.55, 10690950.62]]
{'is_fraud': [False]} False
[[298.0, 1.0, 59395.93, 4127.0, 0.0, 52121.0, 0.0]]
{'is_fraud': [False]} False
[[300.0, 1.0, 192863.59, 0.0, 0.0, 195386.59, 388250.18]]
{'is_fraud': [False]} False
[[396.0, 0.0, 261707.09, 760262.84, 1021969.92, 3732924.34, 3471217.26]]
{'is_fraud': [False]} False
[[276.0, 3.0, 17544.18, 0.0, 0.0, 0.0, 0.0]]
{'is_fraud': [False]} False
[[36.0, 1.0, 298940.1, 31434.0, 0.0, 9291.33, 308231.43]]
{'is_fraud': [False]} False
[[190.0, 1.0, 235993.98, 85510.0, 0.0, 270923.56, 506917.54]]
{'is_fraud': [False]} False
[[10.0, 1.0, 26403.59, 0.0, 0.0, 312966.03, 339369.62]]
{'is_fraud': [False]} False
[[225.0, 4.0, 386545.43, 0.0, 0.0, 1393846.84, 1780392.27]]
{'is_fraud': [False]} False
[[140.0, 3.0, 7004.55, 0.0, 0.0, 0.0, 0.0]]
{'is_fraud

{'is_fraud': [False]} False
[[262.0, 0.0, 68183.35, 720321.58, 788504.93, 2979699.55, 2911516.2]]
{'is_fraud': [False]} False
[[134.0, 0.0, 162614.63, 4396437.04, 4559051.67, 906466.54, 818756.82]]
{'is_fraud': [False]} False
[[209.0, 0.0, 223881.33, 1231858.85, 1455740.18, 253593.52, 29712.19]]
{'is_fraud': [False]} False
[[305.0, 0.0, 287660.33, 9356369.88, 9644030.21, 4512769.83, 4225109.5]]
{'is_fraud': [False]} False
[[228.0, 0.0, 367465.67, 19387.0, 386852.67, 1981713.14, 1614247.46]]
{'is_fraud': [False]} False
[[141.0, 1.0, 57708.51, 0.0, 0.0, 1316830.06, 1161063.25]]
{'is_fraud': [False]} False
[[210.0, 3.0, 5223.32, 5018.0, 0.0, 0.0, 0.0]]
{'is_fraud': [False]} False
[[377.0, 1.0, 103609.56, 0.0, 0.0, 356018.48, 459628.04]]
{'is_fraud': [False]} False
[[304.0, 1.0, 27102.03, 0.0, 0.0, 41104.26, 68206.29]]
{'is_fraud': [False]} False
[[16.0, 4.0, 185768.37, 0.0, 0.0, 189780.04, 58869.44]]
{'is_fraud': [False]} False
[[186.0, 0.0, 18816.37, 8219610.09, 8238426.47, 3194714.3, 31

{'is_fraud': [False]} False
[[633.0, 1.0, 4128853.75, 4128853.75, 0.0, 13982778.91, 18111632.66]]
{'is_fraud': [True]} True
[[236.0, 3.0, 1729.85, 0.0, 0.0, 0.0, 0.0]]
{'is_fraud': [False]} False
[[21.0, 1.0, 282646.94, 0.0, 0.0, 1621855.28, 1904502.22]]
{'is_fraud': [False]} False
[[323.0, 3.0, 38827.16, 0.0, 0.0, 0.0, 0.0]]
{'is_fraud': [False]} False
[[46.0, 3.0, 12892.99, 6680.0, 0.0, 0.0, 0.0]]
{'is_fraud': [False]} False
[[252.0, 3.0, 9139.32, 0.0, 0.0, 0.0, 0.0]]
{'is_fraud': [False]} False
[[402.0, 3.0, 8041.49, 0.0, 0.0, 0.0, 0.0]]
{'is_fraud': [False]} False
[[477.0, 4.0, 338293.14, 338293.14, 0.0, 0.0, 0.0]]
{'is_fraud': [True]} True
[[41.0, 1.0, 12674.45, 243.0, 0.0, 1694809.58, 2169243.83]]
{'is_fraud': [False]} False
[[250.0, 1.0, 64109.83, 38867.0, 0.0, 0.0, 64109.83]]
{'is_fraud': [False]} False
[[398.0, 3.0, 24755.88, 0.0, 0.0, 0.0, 0.0]]
{'is_fraud': [False]} False
[[324.0, 1.0, 93812.46, 9138.48, 0.0, 698036.22, 791848.68]]
{'is_fraud': [False]} False
[[374.0, 1.0, 9

{'is_fraud': [False]} False
[[702.0, 4.0, 573266.42, 573266.42, 0.0, 0.0, 0.0]]
{'is_fraud': [True]} True
[[347.0, 1.0, 280417.88, 0.0, 0.0, 7035868.06, 7316285.94]]
{'is_fraud': [False]} False
[[18.0, 1.0, 172271.14, 0.0, 0.0, 249222.77, 671830.48]]
{'is_fraud': [False]} False
[[139.0, 1.0, 24037.17, 18862.0, 0.0, 4241.67, 28278.84]]
{'is_fraud': [False]} False
[[130.0, 4.0, 2166252.08, 0.0, 0.0, 2609559.03, 4775811.11]]
{'is_fraud': [False]} False
[[401.0, 4.0, 19511.35, 19511.35, 0.0, 0.0, 0.0]]
{'is_fraud': [True]} True
[[694.0, 3.0, 12641.57, 10127.0, 0.0, 0.0, 0.0]]
{'is_fraud': [False]} False
[[158.0, 1.0, 210082.29, 10083.0, 0.0, 1181141.75, 1391224.04]]
{'is_fraud': [False]} False
[[12.0, 1.0, 28158.58, 82693.0, 54534.42, 0.0, 28158.58]]
{'is_fraud': [False]} False
[[187.0, 3.0, 2788.11, 26260.0, 23471.89, 0.0, 0.0]]
{'is_fraud': [False]} False
[[262.0, 4.0, 478594.63, 0.0, 0.0, 591296.5, 1069891.14]]
{'is_fraud': [False]} False
[[135.0, 0.0, 196151.5, 3641460.02, 3837611.52, 

KeyboardInterrupt: 